# Run this before testing anything

In [201]:
import pandas as pd
import numpy as np
import json
import argparse
import os
from datetime import date
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np

from src.selenium_manager import create_stealth_driver
from src.scraping import Scraper
from src.cleaning import DataCleaner, drop_mixed_listings, is_land_only
from src.feature_engineering import FeatureEngineer
from src.address_standardizer import AddressStandardizer
from src import config
from src.utils import save_urls_to_csv, save_details_to_csv, chunks
from src.tasks import scrape_worker
from src.modelling import predict_alley_width

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

def run_cleaning_pipeline():
    """Step 3: Clean the raw data and structure it."""
    if not os.path.exists(config.DETAILS_OUTPUT_FILE):
        print(f"Raw details file not found: {config.DETAILS_OUTPUT_FILE}. Run with `--mode details` first.")
        return

    print(f"Reading raw data from '{config.DETAILS_OUTPUT_FILE}'...")
    df_raw = pd.read_csv(config.DETAILS_OUTPUT_FILE)
    df_raw = drop_mixed_listings(df_raw)

    cleaned_records = []
    for _, row in df_raw.iterrows():
        row_dict = row.to_dict()
        direct_features = DataCleaner.extract_direct_features(row_dict)

        # --- 1. Detect if it is a land-only property ---
        is_land = is_land_only(row_dict)

        # --- 2. Extract all data ---
        processed_data = {
            'Tỉnh/Thành phố': DataCleaner.extract_city(row_dict),
            'Thành phố/Quận/Huyện/Thị xã': DataCleaner.extract_district(row_dict),
            'Xã/Phường/Thị trấn': DataCleaner.extract_ward(row_dict),
            'Đường phố': DataCleaner.extract_street(row_dict),
            'Chi tiết': DataCleaner.extract_address_detail(row_dict),
            'Nguồn thông tin': row_dict.get('url'),
            'Tình trạng giao dịch': 'Rao bán',
            'Thời điểm giao dịch/rao bán': DataCleaner.extract_published_date(row_dict.get('main_info')),
            'Thông tin liên hệ': None,
            'Giá rao bán/giao dịch': DataCleaner.extract_total_price(row_dict.get('main_info')),
            'Loại đơn giá (đ/m2 hoặc đ/m ngang)': 'đ/m2',
            'Số tầng công trình': DataCleaner.extract_num_floors(row_dict),
            'Tổng diện tích sàn': DataCleaner.extract_built_area(row_dict),
            'Đơn giá xây dựng': DataCleaner.get_construction_cost(row_dict),
            'Năm xây dựng': None,
            'Chất lượng còn lại': DataCleaner.estimate_remaining_quality(row_dict),
            'Diện tích đất (m2)': DataCleaner.extract_total_area(row_dict),
            'Kích thước mặt tiền (m)': DataCleaner.extract_facade_width(row_dict),
            'Kích thước chiều dài (m)': DataCleaner.extract_land_length(row_dict),
            'Số mặt tiền tiếp giáp': DataCleaner.extract_facade_count(row_dict),
            'Hình dạng': DataCleaner.extract_land_shape(row_dict),
            'Độ rộng ngõ/ngách nhỏ nhất (m)': DataCleaner.extract_alley_width(row_dict),
            'Khoảng cách tới trục đường chính (m)': DataCleaner.extract_distance_to_main_road(row_dict),
            'Mục đích sử dụng đất': 'Đất ở',
            'Yếu tố khác': " | ".join(direct_features) if direct_features else None,
            'Tọa độ (vĩ độ)': row_dict.get('latitude'),
            'Tọa độ (kinh độ)': row_dict.get('longitude'),
            'Hình ảnh của bài đăng': row_dict.get('image_urls'),
            'description': row_dict.get('description'),
            'is_land': is_land  # <-- Add the temporary flag here
        }

        # --- 3. Apply special logic if it's land only ---
        if is_land:
            processed_data['Số tầng công trình'] = 0
            processed_data['Đơn giá xây dựng'] = 0
            processed_data['Tổng diện tích sàn'] = 0
            processed_data['Chất lượng còn lại'] = 0
        
        cleaned_records.append(processed_data)

    df_cleaned = pd.DataFrame(cleaned_records)

    try:
        # Standardize Province and District using the simplified AddressStandardizer
        address_std = AddressStandardizer(
            config.PROVINCES_SQL_FILE,
            config.DISTRICTS_SQL_FILE,
            config.WARDS_SQL_FILE,
            config.STREETS_SQL_FILE
        )
        df_cleaned['Tỉnh/Thành phố'] = df_cleaned['Tỉnh/Thành phố'].apply(address_std.standardize_province)
        df_cleaned['short_address'] = df_raw['short_address']
        df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        df_cleaned['Xã/Phường/Thị trấn'] = df_cleaned.apply(address_std.standardize_ward, axis = 1)
        df_cleaned.drop(columns=['short_address'], inplace = True)
        df_cleaned.dropna(subset='Diện tích đất (m2)', axis=0, inplace=True)
        return df_cleaned
        # df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        # df_cleaned.drop(columns=['short_address'], inplace=True)
        # print("Province and District standardization complete.")
    except FileNotFoundError:
        print("Skipping province/district standardization because data files were not found.")

cleaned = run_cleaning_pipeline()

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)
# df = df[~(df['Diện tích đất (m2)'] == '')]
df.info()

Reading raw data from 'output/listing_details.csv'...
Removed 4972 listings containing 'thổ cư'.
Error parsing price: local variable 'cleaned_num' referenced before assignment
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27489 entries, 0 to 27488
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27489 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27489 non-null  object 
 2   Xã/Phường/Thị trấn                    27468 non-null  object 
 3   Đường phố                             23800 non-null  object 
 4   Chi tiết                              27489 non-null  object 
 5   url                                   27489 non-null  object 
 6   Tình trạng giao dịch                  27489 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 

In [117]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27489 entries, 0 to 27488
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27489 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27489 non-null  object 
 2   Xã/Phường/Thị trấn                    27467 non-null  object 
 3   Đường phố                             23800 non-null  object 
 4   Chi tiết                              27489 non-null  object 
 5   Nguồn thông tin                       27489 non-null  object 
 6   Tình trạng giao dịch                  27489 non-null  object 
 7   Thời điểm giao dịch/rao bán           27477 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24951 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27489 non-null  object 
 11  Số tầng công tr

# Columns to fix

- Thành phố/Quận/Huyện/Thị xã (Xong)
- Xã/Phường/Thị trấn (xong)
- Giá rao bán/giao dịch (Không check nữa)
- Diện tích đất (m2) (xong)

Đường phố, Số tầng công trình, Tổng diện tích sàn, Đơn giá xây dựng, Chất lượng còn lại, Kích thước mặt tiền (m), Kích thước chiều dài (m), Số mặt tiền tiếp giáp, Hình dạng, Độ rộng ngõ/ngách nhỏ nhất (m), Khoảng cách tới trục đường chính (m)

# Test district from here

In [9]:
import argparse
import os
from datetime import date
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np

from src.selenium_manager import create_stealth_driver
from src.scraping import Scraper
from src.cleaning import DataCleaner, drop_mixed_listings, is_land_only
from src.feature_engineering import FeatureEngineer
from src.address_standardizer import AddressStandardizer
from src import config
from src.utils import save_urls_to_csv, save_details_to_csv, chunks
from src.tasks import scrape_worker
from src.modelling import predict_alley_width

def run_cleaning_pipeline():
    """Step 3: Clean the raw data and structure it."""
    if not os.path.exists(config.DETAILS_OUTPUT_FILE):
        print(f"Raw details file not found: {config.DETAILS_OUTPUT_FILE}. Run with `--mode details` first.")
        return

    print(f"Reading raw data from '{config.DETAILS_OUTPUT_FILE}'...")
    df_raw = pd.read_csv(config.DETAILS_OUTPUT_FILE)
    df_raw = drop_mixed_listings(df_raw)

    cleaned_records = []
    for _, row in df_raw.iterrows():
        row_dict = row.to_dict()
        direct_features = DataCleaner.extract_direct_features(row_dict)

        # --- 1. Detect if it is a land-only property ---
        is_land = is_land_only(row_dict)

        # --- 2. Extract all data ---
        processed_data = {
            'Tỉnh/Thành phố': DataCleaner.extract_city(row_dict),
            'Thành phố/Quận/Huyện/Thị xã': DataCleaner.extract_district(row_dict),
            'Xã/Phường/Thị trấn': DataCleaner.extract_ward(row_dict),
            'Đường phố': DataCleaner.extract_street(row_dict),
            'Chi tiết': DataCleaner.extract_address_detail(row_dict),
            'Nguồn thông tin': row_dict.get('url'),
            'Tình trạng giao dịch': 'Rao bán',
            'Thời điểm giao dịch/rao bán': DataCleaner.extract_published_date(row_dict.get('main_info')),
            'Thông tin liên hệ': None,
            'Giá rao bán/giao dịch': DataCleaner.extract_total_price(row_dict.get('main_info')),
            'Loại đơn giá (đ/m2 hoặc đ/m ngang)': 'đ/m2',
            'Số tầng công trình': DataCleaner.extract_num_floors(row_dict),
            'Tổng diện tích sàn': DataCleaner.extract_built_area(row_dict),
            'Đơn giá xây dựng': DataCleaner.get_construction_cost(row_dict),
            'Năm xây dựng': None,
            'Chất lượng còn lại': DataCleaner.estimate_remaining_quality(row_dict),
            'Diện tích đất (m2)': DataCleaner.extract_total_area(row_dict),
            'Kích thước mặt tiền (m)': DataCleaner.extract_facade_width(row_dict),
            'Kích thước chiều dài (m)': DataCleaner.extract_land_length(row_dict),
            'Số mặt tiền tiếp giáp': DataCleaner.extract_facade_count(row_dict),
            'Hình dạng': DataCleaner.extract_land_shape(row_dict),
            'Độ rộng ngõ/ngách nhỏ nhất (m)': DataCleaner.extract_alley_width(row_dict),
            'Khoảng cách tới trục đường chính (m)': DataCleaner.extract_distance_to_main_road(row_dict),
            'Mục đích sử dụng đất': 'Đất ở',
            'Yếu tố khác': " | ".join(direct_features) if direct_features else None,
            'Tọa độ (vĩ độ)': row_dict.get('latitude'),
            'Tọa độ (kinh độ)': row_dict.get('longitude'),
            'Hình ảnh của bài đăng': row_dict.get('image_urls'),
            'description': row_dict.get('description'),
            'is_land': is_land  # <-- Add the temporary flag here
        }

        # --- 3. Apply special logic if it's land only ---
        if is_land:
            processed_data['Số tầng công trình'] = 0
            processed_data['Đơn giá xây dựng'] = 0
            processed_data['Tổng diện tích sàn'] = 0
            processed_data['Chất lượng còn lại'] = 0
        
        cleaned_records.append(processed_data)

    df_cleaned = pd.DataFrame(cleaned_records)

    try:
        # Standardize Province and District using the simplified AddressStandardizer
        address_std = AddressStandardizer(
            config.PROVINCES_SQL_FILE,
            config.DISTRICTS_SQL_FILE,
            config.WARDS_SQL_FILE,
            config.STREETS_SQL_FILE
        )
        df_cleaned['Tỉnh/Thành phố'] = df_cleaned['Tỉnh/Thành phố'].apply(address_std.standardize_province)
        df_cleaned['short_address'] = df_raw['short_address']
        df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        df_cleaned['Xã/Phường/Thị trấn'] = df_cleaned.apply(address_std.standardize_ward, axis = 1)
        df_cleaned.drop(columns=['short_address'], inplace = True)
        return df_cleaned
        # df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        # df_cleaned.drop(columns=['short_address'], inplace=True)
        # print("Province and District standardization complete.")
    except FileNotFoundError:
        print("Skipping province/district standardization because data files were not found.")

cleaned = run_cleaning_pipeline()

Reading raw data from 'output/listing_details.csv'...
Removed 4972 listings containing 'thổ cư'.
Error parsing price: local variable 'cleaned_num' referenced before assignment


In [ ]:
# from src.address_standardizer import AddressStandardizer
from src import config
import sqlite3
import pandas as pd
from unicodedata import normalize
from rapidfuzz import fuzz

conn = sqlite3.connect(":memory:")
conn.execute("CREATE TABLE provinces (name TEXT, code TEXT, status TEXT);")
conn.execute("CREATE TABLE districts (name TEXT, code TEXT, province_code TEXT, status TEXT);")
conn.execute("CREATE TABLE wards (name TEXT, code TEXT, district_code TEXT, status TEXT);")
conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")

with open(config.PROVINCES_SQL_FILE, "r", encoding="utf-8") as f:
    conn.executescript(f.read())

with open(config.DISTRICTS_SQL_FILE, "r", encoding="utf-8") as f:
    dis_cleaned = f.read().replace("\\'", "''")
    conn.executescript(dis_cleaned)

with open(config.WARDS_SQL_FILE, "r", encoding="utf-8") as f:
    ward_cleaned = f.read().replace("\\'", "''")
    conn.executescript(ward_cleaned)

provinces_df = pd.read_sql_query("SELECT * FROM provinces", conn)
districts_df = pd.read_sql_query("""
    SELECT d.name AS district_name, p.name AS province_name
    FROM districts d
    JOIN provinces p ON d.province_code = p.code
    """, conn)

wards_df = pd.read_sql_query("""
    SELECT w.name AS ward_name,
        d.name AS district_name,
        p.name AS province_name
    FROM wards w
    JOIN districts d ON w.district_code = d.code
    JOIN provinces p ON d.province_code = p.code
""", conn)

if 'conn' in locals():
    conn.close()

reverse_province_map = {
    prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
    for prov in provinces_df['name'].unique()
}

reverse_district = {}
for province in districts_df['province_name'].unique():
    reverse_district[province] = {}
    for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
        district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
        reverse_district[province][district_name_strip] = district_name
reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'
reverse_district['Thành phố Hồ Chí Minh']['Quận 2'] = 'Thành phố Thủ Đức'
reverse_district['Thành phố Hồ Chí Minh']['Quận 9'] = 'Thành phố Thủ Đức'

# for province in districts_df['province_name'].unique():
#     reverse_district[province] = {}
#     for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
#         # district_name_strip = normalize('NFKD', district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip())
#         district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
#         reverse_district[province][district_name_strip] = district_name
# reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'

reverse_ward = {}
for province in reverse_district.keys():
    reverse_ward[province] = {}
    for district in reverse_district[province].values():
        reverse_ward[province][district] = {}
        for ward in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
            ward_name_strip = normalize('NFC', ward.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
            reverse_ward[province][district][ward_name_strip] = ward
# for district in wards_df['district_name'].unique():
#     reverse_ward[district] = {}
#     for ward_name in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
#         ward_name_strip = normalize('NFKD', ward_name.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
#         reverse_ward[district][ward_name_strip] = ward_name

def standardize_district(row):
        prefix = ['Thành phố', 'Thành Phố', 'Quận', 'Huyện', 'Thị xã', 'Thị Xã', 'Đảo']
        district_value = row['Thành phố/Quận/Huyện/Thị xã']
        if isinstance(district_value, str):
            for pre in prefix:
                if district_value.startswith(pre):
                    return district_value
            province = row['Tỉnh/Thành phố']
            if district_value in reverse_district[province].keys():
                return reverse_district[province][district_value]
            for dis in reverse_district[province].keys():
                similarity = fuzz.ratio(district_value, dis)
                if similarity >= 66:
                    print(f'Value: {district_value}')
                    print(f"Short address: {row['short_address']}")
                    print(f"Predicted value: {reverse_district[province][dis]}")
                    print('-' * 50)
                    return reverse_district[province][dis]
            return district_value
        return None

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            try:
                if ward_value in reverse_ward[province_value][district_value].keys():
                    return reverse_ward[province_value][district_value][ward_value]
            except:
                print(f'Ward value: {ward_value}\nDistrict value: {district_value}\nProvince value: {province_value}')
                print('-'*50)
                return None
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add_value = row['short_address']
            if isinstance(short_add_value, str) and short_add_value != '':
                short_add_list = row['short_address'].split(',')
                if len(short_add_list) >= 3:
                    new_province_val = row['Tỉnh/Thành phố']
                    new_ward_val = normalize('NFC',short_add_list[-3].strip())
                    new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                    return matching(new_ward_val, new_district_val, new_province_val)
            return None


# cleaned['ward'] = cleaned.apply(standardize_ward, axis=1)

# Test Ward

In [ ]:
from unicodedata import normalize
import re

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            if ward_value in reverse_ward[province_value][district_value].keys():
                return reverse_ward[province_value][district_value][ward_value]
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
            return None
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add = row['short_address']
            if isinstance(short_add, str) and short_add != '':
                if 'xã' in short_add.lower():
                    print(f"Xã in short_add: {short_add.lower()}")
                    match_result = re.search(pattern='(xã [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'phường' in short_add.lower():
                    print(f'Phường in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(phường [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'thị trấn' in short_add.lower():
                    print(f'Thị trấn in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(thị trấn [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                else:
                    short_add_list = row['short_address'].split(',')
                    if len(short_add_list) >= 3:
                        new_province_val = row['Tỉnh/Thành phố']
                        new_ward_val = normalize('NFC',short_add_list[-3].strip())
                        new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                        return matching(new_ward_val, new_district_val, new_province_val)

            else:
                return None
                # short_add_list = row['short_address'].split(',')
                # if len(short_add_list) >= 3:
                #     new_province_val = row['Tỉnh/Thành phố']
                #     new_ward_val = normalize('NFC',short_add_list[-3].strip())
                #     new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                #     return matching(new_ward_val, new_district_val, new_province_val)
            return None

In [ ]:
from rapidfuzz import fuzz

yo = cleaned['district'].iloc[24895]
dis = list(reverse_district['Tỉnh Đắk Lắk'].keys())[4]
print(fuzz.ratio(yo, dis))

66.66666666666667


# Test Prices (Mức giá)

In [ ]:
import pandas as pd
import numpy as np

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df.info()

In [ ]:
import json

df['other_info'] = df['other_info'].apply(json.loads)
df['price'] = df['other_info'].apply(lambda x: x.get('Mức giá'))

check_price_df = df[~df['price'].str.contains('tỷ', na=False)]
check_price_df = check_price_df[~(check_price_df['price'] == 'Thỏa thuận')]
print(f'Shape: {check_price_df.shape}')

check_price_df.dropna(subset = 'price', inplace=True, axis=0)
check_price_df['digit_price'] = check_price_df['price'].apply(lambda x: x.split()[0].replace(',', '.').strip())
check_price_df['digit_price'] = check_price_df['digit_price'].astype(float)
check_price_df['unit_price'] = check_price_df['price'].apply(lambda x: x.split()[1].strip())
print(check_price_df['unit_price'].unique())

nghin_met_vuong = check_price_df[check_price_df['unit_price'] == 'nghìn/m²']
nghin = check_price_df[check_price_df['unit_price'] == 'nghìn']
trieu_met_vuong = check_price_df[check_price_df['unit_price'] == 'triệu/m²']
trieu_incorrect = check_price_df[(check_price_df['unit_price'] == 'triệu') & (check_price_df['digit_price'] <= 300)]

# Test Khoảng cách tới trục đường chính

In [56]:
import pandas as pd
import numpy as np
import json

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df['other_info'] = df['other_info'].apply(json.loads)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27491 entries, 0 to 27490
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27491 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27491 non-null  object 
 2   Xã/Phường/Thị trấn                    27469 non-null  object 
 3   Đường phố                             23802 non-null  object 
 4   Chi tiết                              27491 non-null  object 
 5   url                                   27491 non-null  object 
 6   Tình trạng giao dịch                  27491 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24953 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27491 non-null  object 
 11  Số tầng công tr

In [57]:
df['Đường vào'] = df['other_info'].apply(lambda x: x.get('Đường vào'))

In [58]:
# df.dropna(subset='Đường vào', axis=0, inplace=True)
df['digit_road'] = df['Đường vào'].apply(lambda x: x.split()[0].replace(',', '.').strip() if x is not None else None)
df['digit_road'] = df['digit_road'].astype(float)

In [59]:
df['Đường vào'].isna().sum()

np.int64(9964)

- Đúng là độ rộng ngõ ngách nhỏ nhất: 135, 319, 1619, 1824, 1962, 2016, 2439, 2532, 2791, 2933, 2988, 3050, 3134, 3142, 3177, 3300, 3367, 3803, 4442, 5387, 5525, 5547, 5724, 6164, 6377, 6699, 6714, 6722, 7029, 8197, 8207, 9092, 9217, 9243, 9336, 10138, 10719, 10998, 11067, 11398, 11703, 12195, 12469, 12901, 13261, 13612, 13799, 13866, 14035, 14153, 14205, 14361, 14671, 15544, 15647, 16204, 16505, 17326, 17427, 17430, 18136, 18450, 19020, 19056, 19178, 19319, 19335, 19761, 19785, 19820, 19865

- Sai: 
    - 410 (50 m): cách bãi gửi ô tô chỉ 50m 
    - 726 (17 m): Lộ giới: 16m
    - 1622 (30 m): Đường trước nhà 4m xe hơi
    - 1956 (40m): Hẻm trước nhà 4m
    - 1988 (40 m): Hẻm ba gác cách mặt tiền 30m
    - 2623 (150 m): Hẻm 5m xe hơi đậu trước nhà
    - 2913 (30 m): Khoảng sân trước 30m tiện kinh doanh mua bán
    - 3312 (200 m): Ngõ rộng 200cm
    - 3374 (50 m): Đường trước nhà 10m Xe hơi vào tận cửa
    - 3375 (50 m): Đường nhựa 10m Xe hơi ra vào thoải mái
    - 3379 (20 m): Đường xe hơi 8m thông thoáng khu vực văn minh
    - 5601 (20 m - khoảng cách tới trục đường chính): Đường trước nhà 4m
    - 6969 (50 m): Nhà cách mặt phố Thái Hà 50m, ngõ rộng 5m
    - 7053 (20 m): cách mặt phố 20
    - 7900 (30 m): Hẻm nhựa 7m xe hơi tận nhà
    - 8521 (22 m): Chính chủ gửi bán căn nhà phố trục chính đường 22m
    - 8692 (60 m): HẺM XE HƠI\nCách mặt tiền chỉ 60m
    - 9268 (17 m): Đường trước nhà 17m. Vỉa hè 3m
    - 9788 (18 m): Mặt tiền đường lớn
    - 9804 (100 m): Hẻm trước nhà 4m
    - 9974 (20 m): mặt tiền đường 20m
    - 10186 (30 m): Đường rộng 30m
    - 10239 (20 m): Tọa lạc tại phố Lê Đức Thọ
    - 10247 + 10251 (16 m): Đường trước nhà: 16m
    - 10448 (20 m): hẻm 10m
    - 10477 (25 m): Đường trước nhà
    - 10566 (24 m): đường trước nhà 24m chạy thẳng ra biển
    - 10654 (16 m): Đường rộng 6m xe tải vào nhà
    - 10746 (16 m): góc đường Thân Văn Nhiếp
    - 10870 (150 m): Ngõ rộng 150, thuận tiện cho xe hơi vào
    - 11114 (16 m): mặt tiền Phổ Quang
    - 11652 (20 m): Đường trước nhà 12m đậu ô tô thoải mái
    - 11774 (20 m): Cần bán gấp nhà Phố Đường số 4 Lò Lu
    - 11950 (30 m): Chính chủ cần bán gấp nhà cấp 4 MT Phạm Văn Bạch, P15, Tân Bình
    - 11959 (23 m): Đường 15m, vỉa hè rộng 5m
    - 12049 (24 m): đường rộng 18m
    - 12110 (150 m): ngõ vào thoáng đãng 150cm
    - 21263 (20 m): Mặt tiền đường lớn 20m, rất thuận tiện trong việc kinh doanh và buôn bán hoặc ở
    - 12292 (30 m): hẻm 5m xe hơi ra vào thoải mái.
    - 12533 (32 m): mặt đường to, vỉa hè cực rộng lên đến 15 mét,
    - 12812 ( 20 m): Nhà nằm trên mặt đường 2 làn xe rộng 20m, có vỉa hè 2 bên
    - 13024 (22 m):  ngõ vào rộng 2.2m
    - 14516 (20 m): đường siêu rộng 20m trước nhà
    - 15171 (18 m): Vị trí là trục đường chính vào KĐT Hạ Đình, Dự Án Rue De Charme,... Nằm cạnh tuyến đường Vành Đai 3 lưu thông các tỉnh.
    - 16058 (30 m): Cách Nguyễn Văn Thoại chỉ 30m.
    - 16436 (24 m): Nhà nằm vị trí đẹp nhất phố Liễu Giai kinh doanh đa dạng.
    - 16443 (17 m): Bán nhà phố thương mại đường Hà Huy Tập, Phú Mỹ Hưng, Quận 7
    - 16498 (16 m): Chính chủ cần bán tòa nhà phố Liễu Giai vỉa hè kinh doanh mặt tiền rộng 14 m.
    - 17199 (22 m): nhà 2 mặt hẻm xe tải 7m
    - 17336 (16 m): phố Ngụy Như Kon Tum
    - 17346 (24 m): phố Ngụy Như Kon Tum
    - 17358 (52 m): Lộ giới đường trước nhà: Rộng 52m
    - 17551 (16 m): Vị trí mặt đường Kinh Doanh Văn Phòng Công Ty tốt.
    - 17736 (30 m): Bán nhà nát đường Pasteur, Quận 3
    - 18916 (20 m): tại đường Nguyễn Lương Bằng - Trần Quang Khải
    - 18936 (28 m): Mặt tiền trải dài 3 hướng: Tây Bắc Đông, tổng mặt tiền lên đến 200m.\nĐường trước đất: Lộ giới 28m.
    - 18937 (40 m):  Mặt đường chính rộng 40m, mặt đường nhánh rộng 21m.
    - 18957 (25 m): Mặt chính đường 25m quy hoạch thông Đà Nẵng - Ngô Quyền, mặt cạnh đường 13.5m
    - 18971 (17 m): lộ giới quy hoạch 17m xe tải tránh nhau thoải mái
    - 18976 (20 m): Vị trí đất nằm trên mặt đường lớn xe tải đi,
    - 19003 (17 m): Bán đất mặt đường 17m 
    - 19022 (30 m): Mặt tiền đường Ống Nước, Dĩ An, Bình Dương
    - 19026 (30 m): Mặt tiền đường 3/2
    - 19027 (60 m): mặt tiền Quốc Lộ 22 rộng 27m, mặt tiền đường Trần Văn Mười rộng 20m
    - 19063 (30 m): BÁN ĐẤT MẶT TIỀN ĐƯỜNG DD5 KDC AN SƯƠNG
    - 19079 (35 m): Mặt đường Nguyễn Tất Thành 35m
    - 19100 (20 m): Đường trước nhà rộng 20m
    - 19106 (200 m): Đất trồng cây lâu năm, cách đường Võ Trần Chí 200m
    - 19108 (200 m): Hẻm xe hơi, chỉ cách Quốc lộ 27 đúng 200m
    - 190109 (34 m): Cần bán 2 lô liền kề ven biển cách Cocobay 4km đường Võ Nguyên Giáp đường 34m lề 5m vệt cây xanh 10m trục đường chính đi Đà Nẵng và Hội An Điện Dương.
    - 19110 (16 m): khu công nghiệp khác trải dài trên lộ giới 60m
    - 19119 (20 m): Nằm đường trục chính số 13
    - 19120 (26,5 m): Đất nền mặt tiền đường 20, Phường Nghi Hòa, Cửa Lò, Nghệ An
    - 19148 (16,5 m): Trục xương sống của khu, đường trước mặt rộng 16,5m
    - 19152 (20,5 m):đường trước nhà rộng 20,5m
    - 19153 (22 m): Chính chủ bán lô đất góc ba mặt đường to rộng 22m
    - 19156 (17 m): Mặt đường 8m,
    - 19164 (62 m): Mặt tiền đường lớn 22m
    - 19179 (17 m): Chiều đường trước nhà: 17m.
    - 19186 (36 m): Mặt tiền Nguyễn Văn Linh TT Phước Hải đường lớn rộng 36m kinh doanh buôn bán
    - 19188 (40 m): mặt tiền Hà Huy Giáp giá tốt nhất Q12.
    - 19198 (20 m): Thửa đất bao quanh 4 mặt phố: Trung Yên 10 - Nguyễn Vĩnh Bảo - Thâm Tâm - Nguyễn Bá Khoản
    - 19200 (20 m): Đất có mặt tiền rộng 9m, mặt đường nhựa 20m thuận lợi kinh doanh
    - 19208 (60m): Mặt tiền tuyến tránh QL56 phường Kim Dinh, Tp Hồ Chí Minh.\n(Phường Kim Dinh, TP Bà Rịa).
    - 19210 (43 m): Lợi thế miếng đất này là 2 mặt tiền. 1 mặt 53m giáp đường chuẩn bị làm đường Quốc Lộ 62 mới
    - 19211 (43 m): Miếng đất lợi thế 2 mặt tiền. 1 mặt tiền sắp chuẩn bị lên đường QL 62 mới mặt tiền rộng 36m.
    - 19215 (28 m): Nhỉnh 6 tỷ có ngay lô đất mặt đường Hải Phong 120m² ( ngang 5m x 24m) vị trí siêu đẹp nằm trên trục chính vinhomes Dương Kinh rộng 28m
    - 19218 (42 m): mặt tiền đường Hùng Vương
    - 19221 (16 m): Chuyên bán đất MT Đường Số P. Tân Quy Q7
    - 19242 (20 m): mặt tiền 119 Quốc Hương, Thảo Điền
    - 19275 (16 m): 2 mặt tiền trước sau cực hiếm, giúp tối ưu thiết kế và ánh sáng tự nhiên, 100% các phòng đều có view và ban công\nMặt tiền chính Đặng Huy Trứ rộng 16m, chỉ vài phút là tới biển
    - 19276 (20 m): mặt tiền đường Hoàng Hoa Thám, P. Quảng Phú, Tp. Quảng Ngãi
    - 19277 (25 m): Siêu VIP MTKD ngang 7.5m Hiệp Bình Chánh sát Gigamall đường 25m có vĩa hè thoáng đang cho thuê ổn định
    - 19279 (17 m): mặt tiền Nguyễn Bá Loan
    - 19291 (46 m): đường lớn ngay ngã tư QL 56
    - 19292 (24 m): Mặt tiền đường D3 rộng 24m
    - 19295 (20 m): mặt đường 323D
    - 19305 (30 m): 2 mặt tiền: Đường phía trước rộng 30m, đường khu dân cư phía sau 8m.
    - 19312 (25 m): Có đường 8m mặt thoáng phía sau, mặt chính đường đôi 25m
    - 19316 (30 m): các thể loại mặt đường Quốc Lộ 3
    - 19321 (16 m): Bán đất mặt tiền đường nhựa Trung Đông 11 xã Thới Tam Thôn huyện Hóc Môn, lộ giới quy hoạch 16m.
    - 19332 (20 m): Cần bán 5 lô mặt tiền trực diện sông Sài Gòn, Đường 20m thuộc phường Hiệp Bình Chánh, khu Gigamall - Phạm Văn Đồng, Thủ Đức
    - 19361 (15,5 m): Trục đường 15,5m 3 xe ô tô tránh nhau
    - 19364 (20 m): 4 mặt tiền với 4 phố bao quanh
    - 19371 (40 m):  mặt đường Lê Quang Đạo kéo dài
    - 19384 (20 m): đường trước đất 20m
    - 19388 (17 m): Đường trước nhà: Trục chính 2 chiều ô tô tránh, container đi thoải mái chạy thẳng ra Linh Đàm.
    - 19403 (42 m): Mặt tiền: HL39 Qh rộng 42m
    - 19411 (20 m): Bán lô góc đường Minh Mạng và đường 10 khu Sơn thủy dt 597m
    - 19423 (20 m): Lô đất 5 tỷ còn sót lại, đường 10m, vỉa hè 5m
    - 19447 (50 m):  Lô đất tọa lạc ngay *ngã tư bùng binh* sầm uất, mặt tiền trục đường chính *Lạc Long Quân 50m*
    - 19467 (20 m): Đất mặt tiền biển và mặt tiền đường chính, mặt tiền biển là 89m mặt tiền đường chính 84m đường rộng 20m
    - 19468 (39 m): Đường vào rộng 39m, thuận tiện cho xe hơi ra vào
    - 19478 (30 m): Trục biển đường 30m 2 làn ô tô, vỉa hè siêu rộng Giao thông huyết mạch trung tâm
    - 19490 (33 m): Lô đất mặt đường QL5 khu vườn hoa đối diện KCN Nomura, TDP Do Nha, phường An Dương, Hải Phòng.\nĐường 32m.
    - 19495 (30 m): Mặt tiền Phạm Văn Đồng trục huyết mạch kết nối nhanh Quận 1
    - 19514 (20 m): Bán cặp đất đường Vương Thừa Vũ, Sơn Trả, Đà Nẵng.
    ??? 19521 (24 m): Lô đất 104m² đất thị trấn bám Quốc lộ 6 tầm 8m sâu 12m, gần cổng KCN Lương Sơn đang vận hành và hoạt động
    - 19523 (30 m): Đất mặt tiền tại Quốc Lộ 13,...Đường vào rộng 30m, dễ dàng tiếp cận
    - 19538 (20,5 m): đường trước rộng, vỉa hè 6m.
    - 19548 (42 m):  lô đất mặt đường 353 Hoà Nghĩa Dương Kinh Hải Phòng
    - 19554 (20 m): Đường vào 20m, đường trước nhà 8m, xe ô tô xe tải ra vào thoải mái, thuận tiệ
    - 19567 (25 m): Đường vào rộng 25m, Vỉa hè rộng 7m. thuận tiện cho việc di chuyển bằng xe ô tô
    - 19580 (24 m): nằm trên trục đường 24m kết nối khu công nghiệp Tam Dương - Lập Thạch
    - 19599 (30 m): Lô đất 4 mặt tiền đã có GPXD
    - 19619 (34 m): Mặt tiền Nguyễn Tất Thành 34m
    - 19622 (30 m): mặt tiền giáp đường lộ giới 30m, mặt hậu giáp sông
    - 19632 (16,5 m): Lô đất nằm tại trục chính đường Lưu Quang Vũ rộng 16.5m
    - 19661 (35 m): vị trí đắc địa trên Quốc Lộ 5A ... đường trước đất 35m, xe cộ lưu thông liên tỉnh
    - 19664 (35 m): Bán đất TMDV mặt tiền biển rộng 35m đường Hoàng Sa - Phù Hợp Xây Khách sạn 
    - 19673 (60 m): mặt đường Lê Hồng Phong.\nMặt tiền 12m, đường sau đất 15m
    - 19675 (60 m): mặt đường Lê Hồng Phong trung tâm TMDV sầm uất Hải Phòng.
    - 19680 (20 m): Đường lớn 20m
    - 19682 (40 m): mặt phố 2 làn đường đôi, vỉa hè bát ngát đá bóng, kinh doanh đỉnh ...  đường rộng 40m, thuận tiện cho xe ô tô ra vào
    - 19693 (30 m): Bán siêu phẩm mặt đường - Aeon Mall Lê Chân, Hải Phòng
    - 19701 (64 m): mặt đường Lê Hồng Phong, thành phố Hải Phòng
    - 19702 (16 m): Bán đất vị trí đẹp mặt đường Vĩnh Lưu tuyến 2 Hoàng Thế Thiện, Lê Hồng Phong.\n- Diện tích: 92m² ngang 4.6m dài 20m.\n- Đường rộng 16m cả vỉa hè.
    - 19718 (60 m):  Mặt tiền đường STX rộng 60m
    - 19727 (16 m): MTKD đường Tỉnh Lộ 10, P. Tân Tạo, Q Bình Tân
    - 19746 (32 m): Mặt tiền đường DH722 nay là Long Hoà, TP HCM
    - 19752 (17 m): mặt tiền đường Xa Lộ Hà Nội ngay ngã tư Thủ Đức ... Đường rộng 17m, vỉa hè 5m rất rộng
    - 19780 (18 m): Đường: Rộng 18m ô tô ra vào thoải mái, thuận tiện kinh doanh, vận chuyển
    - 19786 (42 m): đường vào 42m
    - 19800 (17 m): Đường N5 rộng 17m trải nhựa
    - 19808 (16 m): Đường vào rộng 16m, ô tô ra vào thoải mái.
    - 19826 (30 m): mặt tiền đường Võ Chí Công ... Đường 2 làn rộng 30m
    - 19828 (43 m): Đường vào rộng 43m, dễ dàng cho xe ô tô ra vào thoải mái
    - 19829 (15,5 m): Đường 15,5m vỉa hè 5m
    - 19833 (32 m): Mặt tiền đường lớn 32m 2 làn đường
    - 19838 (20 m ): bán gấp đất mặt đường Củ Chi, ...  Đường nhựa rộng 20m. Là trục đường chính thuận lợi giao thương
    - 19846 (42): Đường nhựa 42m, phía sau 12m, vỉa hè 7m
    - 19855 (25 m): mặt chính đường Phạm Tiến Năng rộng 25m
    - 19865 (20 m): nMặt tiền rộng 10m, đường vào rộng 20m, cực kỳ thoáng đãng
    - 19869 (40 m): MT Trương Văn Bang trung tâm hành chính
    - 19892 (22 m): mặt đường 22m

In [17]:
df.loc[15171]['url']

'https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-xien-phuong-khuong-dinh/mat-ngo-phan-lo-oto-dung-do-dien-tich-50m-mat-tien-5-4m-8-tang-thang-may-gia-27-5ty-pr43208383'

In [60]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

# df[df['digit_road'] > 15][['Đường vào', 'Độ rộng ngõ/ngách nhỏ nhất (m)', 'description']]

- Đường/đường trước nhà 10m, đường nhựa 10m, đường xe hơi 8m, (trục chính) đường 22m, Mặt tiền đường lớn, mặt tiền đường 20m, Đường rộng 30m, Đường trước nhà: 16m, đường rộng 18m, Mặt tiền đường lớn 20m, đường siêu rộng 20m trước nhà, Bán đất mặt đường 17m, đường trước mặt rộng 16,5m, Mặt đường 8m, mặt đường nhựa 20m, Mặt tiền chính Đặng Huy Trứ rộng 16m, mặt chính đường đôi 25m, lộ giới quy hoạch 16m
- Đường trước đất: Lộ giới 28m, Mặt đường chính rộng 40m, Mặt chính đường 25m, lộ giới quy hoạch 17m
- Tọa lạc tại phố Lê Đức Thọ, góc đường Thân Văn Nhiếp, mặt tiền (đường) Phổ Quang, nhà cấp 4 MT Phạm Văn Bạch, mặt đường to, mặt đường 2 làn xe rộng 20m, Nhà nằm vị trí đẹp nhất phố Liễu Giai, Bán nhà phố thương mại đường Hà Huy Tập, tòa nhà phố Liễu Giai, Vị trí mặt đường Kinh Doanh Văn Phòng Công Ty tốt, tại đường Nguyễn Lương Bằng - Trần Quang Khải, Vị trí đất nằm trên mặt đường lớn xe tải đi, Mặt tiền đường Ống Nước, mặt tiền Quốc Lộ 22, Nằm đường trục chính số 13, lô đất góc ba mặt đường, 1 mặt 53m giáp đường chuẩn bị làm đường Quốc Lộ 62 mới, 2 mặt tiền, Siêu VIP MTKD ngang 7.5m Hiệp Bình Chánh sát Gigamall đường 25m có vĩa hè thoáng đang cho thuê ổn định, 

# Test Area (Diện tích đất)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27491 entries, 0 to 27490
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27491 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27491 non-null  object 
 2   Xã/Phường/Thị trấn                    27469 non-null  object 
 3   Đường phố                             23802 non-null  object 
 4   Chi tiết                              27491 non-null  object 
 5   url                                   27491 non-null  object 
 6   Tình trạng giao dịch                  27491 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24953 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27491 non-null  object 
 11  Số tầng công tr

In [190]:
def area(row):
    if row['other_info'] != {}:
        return row['other_info'].get('Diện tích')
    return row['main_info'][1].get('value')

df['area'] = df.apply(area, axis=1)
print(df.shape[0])
df = df[~(df['area'] == '')]
print(df.shape[0])
# df = df[df['area'].str.contains('m²')]
# df['digit_area'] = df['area'].apply(lambda x: float(x.split()[0].replace('.', '').replace(',','.')))

27491
27489


# Test number of floors (Số tầng công trình)

In [1]:
import pandas as pd
import json

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [65]:
df['yes_floor'] = df['other_info'].apply(lambda x: x.get('Số tầng') if x.get('Số tầng') else None)
df['yes_floor'] = df['yes_floor'].apply(lambda x: int(x.split()[0]) if x is not None else None)
print(df[~df['yes_floor'].isna()].shape[0])
no_floor_in_other_info = df[df['yes_floor'].isna()]

16464


In [3]:
import re
import numpy as np
from rapidfuzz import fuzz

index_needed = []

def check_additional_floor(value):
    additional_floor = ['sân thượng', 'sân thương', ' st ', 'trệt', 'trêt', 'tret', 'tum', 'hầm', 'hâm', 'gác lửng', 'gác mái', 'lửng', 'lững', 'lừng']
    result = 0
    for word in additional_floor:
        if word in value:
        # tokens = value.split()
        # for token in tokens:
        #     if fuzz.ratio(token, word) > 70:
            if word == 'sân thượng' or word == 'sân thương':
                additional_floor.remove(' st ')
            if word == ' st ':
                additional_floor.remove('sân thượng')
            print(f'Additional Floor word detected: {word}')
            result += 1
    return result

def clean_num_floor(row):
    print(f'Cleaning for row {row["index"]}')
    floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    # reuse word_to_num from before
    num_words_pattern = "|".join(sorted(word_to_num.keys(), key=lambda x: -len(x)))
    # ------- TH1: Thông tin đã có sẵn ở other_info ------
    if row['other_info'] != {} and row['other_info'].get('Số tầng'):
        return int(row['other_info'].get('Số tầng').split()[0])
    # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
    if pd.notna(row['title']):
        lower_title = row['title'].lower()
        old_house = re.search(pattern=r'nhà (?:\w+\s){0,5}cũ', string=lower_title)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_title)
        if cap4:
            return 1
    if pd.notna(row['description']):
        try:
            lower_des = row['description'].lower()
            old_house = re.search(pattern=r'nhà (?:\w+\s){0,5}cũ', string=lower_des)
            if old_house:
                return 0
            cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
            if cap4:
                return 1
        except:
            print(row['description'] == np.nan)
    # ------ TH3: Xét số tầng ------
    # Tống số tầng trong title
    if pd.notna(row['title']):
        add_key = check_additional_floor(lower_title)
        for keyword in floor_keywords:
            if keyword in lower_title:
                num_floor = re.search(pattern=rf'(\d|{num_words_pattern})\s*{keyword}', string=lower_title)
                if num_floor:
                    print(f'Extracted floor in title: {num_floor.group(1)}')
                    print(f'Additional value: {add_key}')
                    if num_floor.group(1).isdigit():
                        possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*{keyword}', string=lower_title)
                        if possible_float:
                            print(f'Found possible float: {possible_float.group(1)}')
                            return float(possible_float.group(1).replace(',','.')) + add_key
                        return int(num_floor.group(1)) + add_key
                    return word_to_num[num_floor.group(1)] + add_key
                    # elif num_floor.group(1) in word_to_num.keys():
                    #     index_needed.append(row['index'])
                    #     return word_to_num[num_floor.group(1)] + add_key
    if pd.notna(row['description']):
        add_key = check_additional_floor(lower_des)
        # Trong trường hợp nêu rõ tầng 1, tầng 2,... thì max sẽ là tổng số tầng
        total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
        if total_pattern:
            print(f"Extracted total floor in description: {total_pattern}")
            total_floor_num = []
            for digit in total_pattern:
                if digit.isdigit():
                    total_floor_num.append(int(digit))
                elif digit in word_to_num.keys():
                    total_floor_num.append(word_to_num[digit])
            if total_floor_num:
                return max(total_floor_num)
        
        # Trong trường hợp liệt kê ra cả lố tầng thì là cộng tổng vào
        separate_pattern = re.search(pattern=rf'(\d|{num_words_pattern})\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        if separate_pattern:
            print(f'Extracted floor that needs to be sum up: {separate_pattern.group(1)}')
            print(f'Additional value: {add_key}')
            if separate_pattern.group(1).isdigit():
                possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
                if possible_float:
                    print(f'Found possible float: {possible_float.group(1)}')
                    return float(possible_float.group(1).replace(',','.')) + add_key
                return int(separate_pattern.group(1)) + add_key
            return word_to_num[separate_pattern.group(1)] + add_key
    return None
    # elif row['description'] is not None:
    #     lower_des = row['description'].lower()
    #     old_house = re.search(pattern=rf'nhà [\w+\s]{0-5}cũ', string=lower_des)
    #     if old_house:
    #         return 0
    #     cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
    #     if cap4:
    #         return 1
        
    #     add_key = check_additional_floor(lower_des)

# df['index'] = df.index
# df['floor'] = df.apply(clean_num_floor, axis = 1)
# error_floor = df[df['floor'].isna()]

In [7]:
error_floor['floor'] = error_floor.apply(clean_num_floor, axis = 1)

Cleaning for row 3
Cleaning for row 36
Cleaning for row 53
Additional Floor word detected: trệt
Additional Floor word detected: trệt
Cleaning for row 62
Additional Floor word detected: trệt
Additional Floor word detected: hầm
Extracted floor that needs to be sum up: 3
Additional value: 2
Cleaning for row 68
Extracted floor that needs to be sum up: 6
Additional value: 0
Cleaning for row 74
Cleaning for row 82
Cleaning for row 184
Additional Floor word detected: sân thượng
Additional Floor word detected: trệt
Extracted floor that needs to be sum up: 2
Additional value: 2
Cleaning for row 193
Extracted floor that needs to be sum up: 6
Additional value: 0
Cleaning for row 196
Cleaning for row 203
Cleaning for row 204
Additional Floor word detected: tum
Extracted total floor in description: ['1', '4', '5']
Cleaning for row 207
Extracted total floor in description: ['1', '2', '6']
Cleaning for row 238
Cleaning for row 243
Cleaning for row 259
Cleaning for row 263
Cleaning for row 282
Extract

C:\Users\Admin\AppData\Local\Temp\ipykernel_16460\1502755388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_floor['floor'] = error_floor.apply(clean_num_floor, axis = 1)


In [ ]:
# list_index = [207, 593, 756, 1235, 1282, 1451, 2099, 2122, 2365, 2691, 2700, 2755, 2764, 2784, 2800, 3185, 3883, 4085, 4538, 5041, 5150, 5212, 5525, 5742, 5790, 5900, 6168, 6445, 6602, 6635, 6718, 6725, 6801, 6963, 7017, 7024, 7050, 7063, 7276, 7609, 8344, 8672, 8723, 8836, 9123, 9127, 9363, 9367, 9548, 9879, 9945, 9955, 10166, 10193, 10291, 10530, 10683, 11726, 11767, 12106, 12339, 12571, 12646, 12744]
# error_floor = error_floor.loc[list_index]
# error_floor['floor'] = error_floor.apply(clean_num_floor, axis = 1)

756: giấy phép xây dựng
1282: cộng thêm sân thượng trong khi nó không có
1451: cái này đúng nhưng mà là để note lại số words có thể có giữa số tầng và các additional floor (Toà nhà 9 tầng, thông sàn, có hầm để xe. )
4538: 3 tầng, 1 tum (nhưng lúc trước nó đã có đề cập tới 4 tầng rồi)
5212: Kết cấu 4 tầng gồm 1 trệt , 2 lầu , sân thượng (mà nó luận ra thành 6 tầng)
5525: 9 - 10 tầng (mà nó extract ra 0 tầng)
5900:  Xây mới 8 tầng thang máy, tổng 13 phòng khép kín cao cấp, full nội thất.\nDoanh thu 80 triệu/tháng lãi ròng ~800 triệu/năm.\n- Tầng 1 thông sàn để xe + WC, các tầng trên 2 phòng rộng rãi, có ban công riêng, máy giặt riêng từng phòng.\n- PCCC đầy đủ: Cửa thép, bình chữa cháy, hệ thống dập lửa tự động, tum thoát hiểm. (extract thành 9)
6725:  trệt 2 lầu ST (extract là 3 tầng)
7024: Kết cấu: Nhà 1 tầng đã cũ thuận tiện xây mới (extract thành 2)
7276: Hiện trạng: 2 tầng + sẵn sàng khai thác hoặc xây mới (cộng thêm 1 cái hầm)
9548: Kết cấu: 1 trệt 2 lầu gồm 4 PN 3 toilet phòng thờ, phòng giặt, sân thượng (extract thành 4 tầng)
10291: Kết cấu: 1 trệt 2 lầu (mà extract thành 4 do có cái sân thượng ở tít đằng sau)
10530: có nhà 2 tầng cũ (extract thành 2)


### New
- 443: Kết cấu: 2 tầng (scrape thành 4 tầng)
- 460: Nhà xây mới 1 trệt 1 lầu đúc (vấn đề là có 1 cái sân thượng ở đằng sau)
- 552: thiết kế 5 tầng (scrape thành 1 tầng)
- 748: 1 trệt 1 lầu đúc (scrape thành 3 tầng do có cộng thêm 1 cái sân thượng ở dưới)

--> Có vẻ vấn đề là phải dính cấu trúc các phòng vào thành 1

### Xét các dòng bị NaN
- Kết cấu : trệt lầu st (trong title còn có trệt lầu, st)
- Thật sự không ghi rõ bao nhiêu tầng

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

error_floor[error_floor['floor'].isna()][['description', 'floor', 'title']].iloc[:100]

,description,floor,title
3,"Bán nhà Phố Nguyễn Khánh Toàn, Cầu Giấy, khu phân lô, 82m², ở yên tĩnh.\n+ Nằm trong khu phân lô quân đội, ô tô vào tận cửa, khu vực dân trí cao, an ninh tốt.\n+ Giao thông di chuyển thuận tiện mọi nẻo đường, kết nối với Nguyễn Khánh Toàn, Nguyễn Đình Hoàn, Quan Hoa, Nguyễn Văn Huyên, Trần Cung, Đội Cấn,\nĐào Tấn, Bưởi, Trần Đăng Ninh, Nguyễn Phong Sắc,...\n+ Gần các trường cấp 1-2 Dịch Vọng, Công viên Nghĩa Đô, Trung tâm Thương mại Lotte,...\n+ Nhà dân xây chắc chắn, phù hợp cho thuê hoặc ở hộ gia đình đều được.\n+ Sổ đỏ đẹp, pháp lý rõ ràng, sẵn sàng giao dịch.\n+ Giá 18.2 tỷ (CÓ THƯƠNG LƯỢNG).\n\n***LH ngay để được tư vấn và hỗ trợ:\nMr. Thao\n0982 609 ***\n(miễn trung gian, môi giới).",NaN,"Bán nhà 82m2 18.2 tỷ, phân lô, đường Nguyễn Khánh Toàn, Cầu Giấy, ở yên tĩnh,"
36,"46m² nhà đẹp kiên cố, ô tô đỗ cổng, giá chỉ 6, x tỷ, Phương Canh.\n\n- Vị trí cực đẹp: Ngõ nông rộng thoáng, ô tô đỗ ngay nhà, cách phố chỉ vài bước chân.\n- Chủ tự xây vô cùng chắc chắn, thiết kế hiện đại, công năng đầy đủ, nội thất tặng lại cho khách thiện chí.\n- Sổ đẹp, chính chủ, giao dịch ngay.\n- Cam kết thông tin chuẩn 100%.\n\nLiên hệ ngay em Yên\n0337 683 ***\nđể xem nhà trực tiếp.",NaN,"46m2 nhà đẹp kiên cố, ô tô đỗ cổng, giá chỉ 6,x tỷ, Phương Canh - vị trí cực đẹp: Ngõ nông rộng th"
53,"Chủ nhà cần bán hẻm xe hơi quay đầu Lê Văn Sỹ ,P12,Q3, hẻm thẳng , cách mặt tiền 15m\nDiện tích : 5,9x12m (71m²)\nKết cấu : trệt lầu st\nNhà mới ở liền , không lỗi phong thuỷ , hẻm thông\nChủ nhà thiện chí bán , hàng xóm lịch sự , khu dân trí cao\nGiá : 12Tỷ 9 TL\nLiên hệ :\n0933 572 ***\nĐức Vinh",NaN,"Hẻm xe hơi quay đầu Lê Văn Sỹ,P12,Q3 ,5.9x12(CN 71m2),trệt lầu,St, 12.9 Tỷ TL"
74,"+ Gần ngay ngã tư giao cắt giữa trung tâm khu vip đi đâu cũng tiện. Nằm trong ngõ 152 Võ Chí Công là khu phân lô cực kì đẹp của Quận Tây Hồ.\n+ Đường trước nhà 2 ô tô tránh nhau thoải mái. Có thoáng trước thoáng sau.\n+ Diện tích 70m².\n+ Sổ đỏ chủ không vay mượn, sẵn sàng giao dịch.\n+ Giá 19.5 tỷ còn thương lượng.\nLiên hệ ngay: Mr. Trung-\n0973 538 ***\nđể xem nhà và chốt deal nhanh chóng!",NaN,"Bán nhà xe hơi tránh ngõ 152 Võ Chí Công - TT - Bộ Giáo Dục. DT: 70m2 MT: 6m, giá 19.5 tỷ"
82,Nhà riêng giấy tờ pháp lý đầy đủ.\nVị trí đẹp.\nTiện kinh doanh buồn bán hàng tạp hóa.\nDiện tích 70m².\nGiá 60tr/m².\nLH\n0969 026 ***\n.,NaN,Nhà chính chủ cần bán miễn tiếp cò
196,HÀNG HOTDÃY TRỌ 156M2 ĐANG CÓ THU NHẬP 45 TRIỆU/THÁNG - GẦN ĐÌNH PHONG PHÚ - TĂNG NHƠN PHÚ B - QUẬN 9\n\n- Diện tích 156m²\n- Tiện ích đầy đủ gần chợ trường học các cấp\n- Giá bán 12 tỷ TL\n\nLH\n0909 675 ***,NaN,Dãy Trọ 156m2 đang cho thu nhập 45 triệu/tháng Đình Phong Phú Quận 9
203,"Phước Long B không có căn thứ 2 gần 100m² chủ đang chào 6,8tỷ TL.\n\n- DT: 94m² ngang 4,5m ô tô ngủ trong nhà.\n- Vị trí: Cách 50m ra Đỗ Xuân Hợp - Trường học các cấp - Chợ Phước Long B.\n- Cách ngã tư Bình Thái - MeTro chỉ 300m.\n- Nhà đẹp lung linh mua về ở ngay.\n- Gồm 3PN, 2WC.\n\nLH\n0909 675 ***\n.",NaN,"Phước Long B duy nhất một căn gần 100m2 chỉ 6,8 tỷ"
238,"Nhà riêng xịn xò tại Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí Minh.\nVới diện tích 60m².\n\n+ Pháp lý đầy đủ, an tâm khi giao dịch.\n+ Không gian sống thoáng đãng và tiện nghi.\n\nKhu vực này cực kỳ thuận tiện cho cuộc sống hàng ngày.\n+ Gần trường học.\n+ Gần chợ/siêu thị.\n+ Có công viên.\n\nLiên hệ\n0936 066 ***\nLan Ngọc.",NaN,"Bán nhà tại Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí Minh, 60 m2 với giá tốt"
243,"Nhà riêng tại Nguyễn Tuyển, Phường Bình Trưng Tây, Thủ Đức, Hồ Chí Minh là một cơ hội tuyệt vời dành cho những ai đang tìm kiếm một không gian sống thoải mái và tiện nghi.\n\n- Diện tích: 95m².\n- 1 phòng ngủ (PN).\n- 1 phòng tắm (WC).\n- Không nội thất.\n- Hướng cửa chính: Tây Nam.\n- Hướng ban công: Nam.\n- Mặt tiền rộng: 8m.\n- Đường vào: 10m.\n- Giá: 14 tỷ VND.\n- Pháp lý đầy đủ.\n\nĐiểm đặc sắc:\n- Phong thủy tốt với hướng cửa chính Tây Nam, mang lại sự thuận lợi cho gia chủ.\n- Mặt tiền rộng phù hợp cho các hoạt động 

In [38]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

# no_floor[~no_floor['floor'].isna()][['description', 'floor']]

In [27]:
no_floor = df[df['floor'].isna()]
no_floor.shape

(9417, 37)

In [85]:
df[(~df['floor'].isna()) & (df['other_info'] != {})][['floor', 'title']]

,floor,title


In [81]:
df[(~df['floor'].isna()) & (df['other_info'] != {}) & (df['other_info'].apply(lambda x: x.get('Số tầng') is None)) & (df['floor'] > 1)][['other_info', 'floor', 'title']]
# sai 482, 

,other_info,floor,title


In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

no_floor_in_other_info[no_floor_in_other_info['title'].str.contains('xây dựng')]['title']

6642     Bán nhà Bình Thạnh, Phường 13, Bình Lợi - Đặng Thùy Trâm (6x21m) xây dựng mới 4 tầng chỉ 13.9 tỷ TL
18950                                                Chỉ hơn 2 tỷ có ngay 200m2 sổ riêng xây dựng đất Đà Lạt
19348                 Bán đất xây nhà xưởng gần KCN Giang Điền, Đồng Nai, pháp lý đầy đủ, giấp phép xây dựng
19408                                   Đất Đặng Văn Bi - 90m2(5x18m) - đường nhựa 8m có lề - xây dựng tự do
19469         Bán lô đất Quốc Lộ 50 đường Bờ Bao, xã Phong Phú, Bình Chánh - xây dựng tự do - Sổ đỏ CN, 6tỷ3
19622                         Đất SHR pháp lý sạch xây dựng tự do Giá F0 Tân Phú Trung Củ Chi 8x10m full thổ
19784       Siêu rẻ siêu đắc địa gần 500m2 xây dựng Villa, KS khu vực tập trung nhiều khách du lịch ở Hội An
19786                                   Bán lô đất mặt tiền 159,7m2 đường số 1 Trần Não xây dựng 1 hầm 4 lầu
19805                  Dịch vụ xê Nam Ninh 50m2, đã có giấy phép xây dựng, lô nguyên 1 chủ. Giá nhỉnh 8 tỷ x
19924      Bán đất 

In [ ]:
# def get_num_floor(row):
#     if 'Số tầng' in row['other_info']:
#         return int(row['other_info'].get('Số tầng').split()[0])
#     return None

# if isinstance(df.iloc[0]['other_info'], str):
#     df['other_info'] = df['other_info'].apply(json.loads)
# if isinstance(df.iloc[0]['main_info'], str):
#     df['main_info'] = df['main_info'].apply(json.loads)
# df['floor'] = df.apply(get_num_floor, axis=1)

In [234]:
all_dict_keys = set()

for row_index in range(df.shape[0]):
    for i in range(len(df['main_info'].iloc[row_index])):
        all_dict_keys.add(df.iloc[row_index]['main_info'][i].get('title'))

print(all_dict_keys)

{'Ngày đăng', '', 'Ngày hết hạn', 'Mức giá', 'Diện tích', 'Mã tin', 'Loại tin', 'Phòng ngủ'}


In [14]:
test = df[(df['floor'].isna() == False) & (df['floor'] != df['Số tầng công trình'])][['description', 'Số tầng công trình', 'floor', 'address_parts']]
test['Số tầng công trình'].unique()

array([0])

Các trường hợp trên xảy ra đều là vì cái part is_land, tức là những cái nào mà chủ ghi bán đất, bán lô đất, bán nền đất hoặc bán đất nền đều sẽ bị auto set num_floor về 0

- hầm, tum, sân thượng đều tính thẳng thành 1 tầng
- giữa số tầng hiện tại và số tầng trong giấy phép xây dựng (hiện trạng là nhà 2 tầng cũ và sân vườn, thuận tiện phá dỡ xây mới, đã có Giấy phép xây dựng 8 tầng nổi 1 hầm), lấy số tầng 0 với nhà cũ
- các loại bán đất, bán đất tặng nhà,... (Nhà C4, xác định bán đất - tại Phường Mỗ Lao, Hà Đông, Hà Nội, với diện tích 68m², giá 10,5 tỷ VND) thì ghi số tầng là 0
- các số tầng cộng thường sẽ ghi dính với nhau thành 1 cụm

- Các trường hợp đặc biệt:
    - nhà cấp 4, cấp 4, nc4, nhà c4, nhà trệt: 1
    - nhà cũ, tiện xây mới: 0
